In [1]:
import pandas as pd 
import numpy as np 
import re

In [2]:
df=pd.read_csv('../data/problems.csv')
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2\n2 3\n1 3\n3...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0\n2 3 2\n50 60 50\n30 50 40', 'ou...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,NaN,NaN,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3\n3 C\n2 C\n1 C', 'output': 'GH...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0\n10 0\n10 10', 'output': '14.1...",hard,9.6,https://open.kattis.com/problems/barktree


In [3]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4031 non-null   object 
 2   input_description   3992 non-null   object 
 3   output_description  3981 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB


title                   0
description            81
input_description     120
output_description    131
sample_io               0
problem_class           0
problem_score           0
url                     0
dtype: int64

In [4]:
df=df.fillna('')

In [5]:
df = df.drop(columns=["sample_io", "url"])

In [6]:
df["full_text"] = (
    df["title"] + " " +
    df["description"] + " " +
    df["input_description"] + " " +
    df["output_description"]
)

In [7]:
import re
df["full_text"]=df["full_text"].str.lower()
df["full_text"] = df["full_text"].apply(
    lambda x: re.sub(r"[^a-z0-9 ]", " ", x)
)
df["full_text"] = df["full_text"].apply(
    lambda x: " ".join(x.split())
)

In [8]:
df[["full_text", "problem_class", "problem_score"]].head()

,full_text,problem_class,problem_score
0,uuu unununium uuu was the name of the chemical...,hard,9.7
1,house building a number of eccentrics from cen...,hard,9.7
2,mario or luigi mario and luigi are playing a g...,hard,9.6
3,the wire ghost ofka is bending a copper wire s...,hard,9.6
4,barking up the wrong tree your dog spot is let...,hard,9.6


In [9]:
df["full_text"].isnull().sum()

np.int64(0)

In [10]:
X = df["full_text"]
y_class = df["problem_class"]
y_score = df["problem_score"]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_class_train, y_class_test, y_score_train, y_score_test = train_test_split(
    X,
    y_class,
    y_score,
    test_size=0.2,
    random_state=42,
    stratify=y_class 
)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=12000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    stop_words="english",
    sublinear_tf=True
)

train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)


In [14]:
train_vectors.shape, test_vectors.shape

((3289, 12000), (823, 12000))

In [15]:
train_text = X_train
test_text = X_test

train_labels = y_class_train
test_labels = y_class_test

In [16]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [17]:
difficulty_model = LinearSVC(class_weight="balanced")
difficulty_model.fit(train_vectors, train_labels)

,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseud

In [18]:
predicted_labels = difficulty_model.predict(test_vectors)

In [19]:
accuracy = accuracy_score(test_labels, predicted_labels)
accuracy

0.4726609963547995

In [20]:
df["problem_class"].value_counts(normalize=True)

problem_class
hard      0.472033
medium    0.341683
easy      0.186284
Name: proportion, dtype: float64

In [21]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(
    max_iter=3000,
    class_weight="balanced",
    C=0.5,          
    n_jobs=-1
)

log_model.fit(train_vectors, train_labels)

log_preds = log_model.predict(test_vectors)
log_acc = accuracy_score(test_labels, log_preds)
log_acc


c:\Users\laksh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


0.4957472660996355

In [22]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(
    class_weight="balanced",
    C=0.5
)

svm_model.fit(train_vectors, train_labels)

svm_preds = svm_model.predict(test_vectors)
svm_acc = accuracy_score(test_labels, svm_preds)
svm_acc

0.48724179829890646

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

nb_vectorizer = TfidfVectorizer(
    max_features=12000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    stop_words="english"
)

nb_train_vectors = nb_vectorizer.fit_transform(train_text)
nb_test_vectors = nb_vectorizer.transform(test_text)

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

nb_model = MultinomialNB(alpha=0.7)   
nb_model.fit(nb_train_vectors, train_labels)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",0.7
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [25]:
nb_preds = nb_model.predict(nb_test_vectors)

nb_accuracy = accuracy_score(test_labels, nb_preds)
nb_accuracy

0.4896719319562576

In [26]:
final_classifier = log_model
final_classifier_accuracy = log_acc

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_labels, log_preds)

array([[ 86,  30,  37],
       [ 63, 229,  97],
       [ 57, 131,  93]])

In [28]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, log_preds))

              precision    recall  f1-score   support

        easy       0.42      0.56      0.48       153
        hard       0.59      0.59      0.59       389
      medium       0.41      0.33      0.37       281

    accuracy                           0.50       823
   macro avg       0.47      0.49      0.48       823
weighted avg       0.50      0.50      0.49       823



In [29]:
from sklearn.metrics import f1_score

macro_f1 = f1_score(test_labels, log_preds, average="macro")
macro_f1

0.4777278717114959

In [30]:
train_scores = y_score_train
test_scores = y_score_test

In [31]:
from sklearn.ensemble import RandomForestRegressor

score_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

score_model.fit(train_vectors, train_scores)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [32]:
predicted_scores = score_model.predict(test_vectors)

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(test_scores, predicted_scores)
rmse = np.sqrt(mean_squared_error(test_scores, predicted_scores))

mae, rmse

(1.699537363304982, np.float64(2.028197700194016))

In [34]:
list(zip(test_scores[:10], predicted_scores[:10]))

[(6.3, np.float64(5.241499999999998)),
 (5.0, np.float64(5.171)),
 (2.0, np.float64(5.479000000000002)),
 (2.4, np.float64(4.778000000000002)),
 (3.0, np.float64(4.4065)),
 (3.0, np.float64(5.035)),
 (1.8, np.float64(4.684499999999999)),
 (5.7, np.float64(5.835000000000001)),
 (8.7, np.float64(6.057000000000001)),
 (3.7, np.float64(4.1475000000000035))]

In [35]:
np.corrcoef(test_scores, predicted_scores)[0, 1]

np.float64(0.40525539321501175)

In [36]:
check_df = pd.DataFrame({
    "actual_score": test_scores,
    "predicted_score": predicted_scores,
    "difficulty": test_labels.values
})

check_df.groupby("difficulty")[["actual_score", "predicted_score"]].mean()

,actual_score,predicted_score
difficulty,,
easy,1.988235,4.323959
hard,7.111311,5.191787
medium,4.134164,4.947743


In [37]:
check_df.groupby("difficulty")["predicted_score"].mean().sort_values()

difficulty
easy      4.323959
medium    4.947743
hard      5.191787
Name: predicted_score, dtype: float64

In [38]:
import joblib

In [39]:
joblib.dump(vectorizer, "../models/text_vectorizer.pkl")

['../models/text_vectorizer.pkl']

In [40]:
joblib.dump(log_model, "../models/difficulty_classifier.pkl")

['../models/difficulty_classifier.pkl']

In [41]:
joblib.dump(score_model, "../models/score_regressor.pkl")

['../models/score_regressor.pkl']